In [50]:
import pandas as pd
import numpy as np

import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor, wait, ALL_COMPLETED

In [3]:
data = pd.read_csv('train_ver2.csv')

C:\Users\Maxim\AppData\Local\Temp/ipykernel_41064/4221001459.py:1: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('train_ver2.csv')


In [4]:
data.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [5]:
data.columns

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov',
       'nomprov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'],
      dtype='object')

In [6]:
print(data['fecha_dato'].value_counts())
print(data['fecha_dato'].mode())

2016-05-28    931453
2016-04-28    928274
2016-03-28    925076
2016-02-28    920904
2016-01-28    916269
2015-12-28    912021
2015-11-28    906109
2015-10-28    892251
2015-09-28    865440
2015-08-28    843201
2015-07-28    829817
2015-06-28    632110
2015-05-28    631957
2015-04-28    630367
2015-03-28    629209
2015-02-28    627394
2015-01-28    625457
Name: fecha_dato, dtype: int64
0    2016-05-28
Name: fecha_dato, dtype: object


## Product names

In [7]:
product_names = {"ind_ahor_fin_ult1" : "Saving Account",
"ind_aval_fin_ult1" : "Guarantees",
"ind_cco_fin_ult1" : "Current Accounts",
"ind_cder_fin_ult1" : "Derivada Account",
"ind_cno_fin_ult1" : "Payroll Account",
"ind_ctju_fin_ult1" : "Junior Account",
"ind_ctma_fin_ult1" : "Más Particular Account",
"ind_ctop_fin_ult1" : "Particular Account",
"ind_ctpp_fin_ult1" : "Particular Plus Account",
"ind_deco_fin_ult1" : "Short-term Deposits",
"ind_deme_fin_ult1" : "Medium-term Deposits",
"ind_dela_fin_ult1" : "Long-term Deposits",
"ind_ecue_fin_ult1" : "E-account",
"ind_fond_fin_ult1" : "Funds",
"ind_hip_fin_ult1" : "Mortgage",
"ind_plan_fin_ult1" : "Pensions",
"ind_pres_fin_ult1" : "Loans",
"ind_reca_fin_ult1" : "Taxes",
"ind_tjcr_fin_ult1" : "Credit Card",
"ind_valo_fin_ult1" : "Securities",
"ind_viv_fin_ult1" : "Home Account",
"ind_nomina_ult1" : "Payroll",
"ind_nom_pens_ult1" : "Pensions",
"ind_recibo_ult1" : "Direct Debit"}

def change_names(col_names, map_products=product_names):
    '''
    Change column names (e.g."ind_recibo_ult1") to product names (e.g."Direct Debit").
    '''
    return list(map(lambda col_name: map_products[col_name], col_names))

In [8]:
USER_ITEM_COLS = [
    'ncodpers',
    'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
    'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
    'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
    'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
    'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
    'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
    'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
    'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'
]
ITEM_COLS = [
    'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
    'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
    'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
    'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
    'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
    'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
    'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
    'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'
]

In [9]:
df_train_0716 = data[data.fecha_dato == '2015-07-28']
df_test_0816 = data[data.fecha_dato == '2015-08-28']

## Popular items

In [10]:
def popularity_based(df):
    """
    Function that calculates the probability of a product occurring. 
    Probability range is <0, 1>.
    """
    top_col = {}
    df = df[ITEM_COLS]
    for col in df.columns:
        top_col[col] = df[col].value_counts()[1]
        
    # sorted by most popular
    top_col = dict(sorted(top_col.items(), key=lambda it: it[1], reverse=True)) 
    
    for k, v in top_col.items():
        top_col[k] = np.around(v / df.shape[0], decimals=4)
        
    return top_col

## Hybrid recommends

In [11]:
def hybrid_recommend(df):
    popularity_outcome = popularity_based(df)
    return popularity_outcome

## Recommendation for one user

In [12]:
def recommendation(user_id, df, hybrid_outcome):
    """
    Function that returns a list of recommendations for a given user.
    """
        
    # products that the user already owns
    df = df[USER_ITEM_COLS]
    user_row = df[df.ncodpers == user_id]
    user_products = list(filter(lambda product: user_row[product].to_numpy()[0]==1, user_row))
                
    # removes products that the user already owns
    recom = { key : hybrid_outcome[key] for key in hybrid_outcome if key not in user_products}

    recom_sort = dict(sorted(recom.items(), key=lambda item: item[1], reverse=True))
    
    return list(recom_sort.keys())

In [13]:
user_id = 1050613

hybrid_outcome = hybrid_recommend(df_train_0716)
rec = recommendation(user_id, df_train_0716, hybrid_outcome)

In [14]:
change_names(rec)

['Current Accounts',
 'Particular Account',
 'Direct Debit',
 'E-account',
 'Payroll Account',
 'Pensions',
 'Payroll',
 'Taxes',
 'Credit Card',
 'Particular Plus Account',
 'Long-term Deposits',
 'Securities',
 'Funds',
 'Junior Account',
 'Pensions',
 'Más Particular Account',
 'Mortgage',
 'Home Account',
 'Loans',
 'Short-term Deposits',
 'Medium-term Deposits',
 'Derivada Account',
 'Saving Account',
 'Guarantees']

## Metrics

In [15]:
# referring to https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py

def apk(actual, predicted, k=7):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=7):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

## Evaluation

In [32]:
one = np.array([1,2,3,4])
two = np.array([1,0,1,0])
one[two.astype(bool)]

array([1, 3])

In [34]:
def rec_test(user_id, df_train, df_test):
    """
    Function that returns a list of test recommendations for a given user.
    """
    actual, recom_test = [], []
    data_after = np.array(df_test[df_test.ncodpers == user_id][ITEM_COLS].values[0])
    try:
        data_before = np.array(df_train[df_train.ncodpers == user_id][ITEM_COLS].values[0])
    except IndexError:
        return df_train[ITEM_COLS].columns[data_after.astype(bool)]
    
    
    for idx, col in enumerate(df_train[ITEM_COLS].columns):    
        if data_before[idx] == 0 and data_after[idx] == 1:
            recom_test.append(col)

    return recom_test

In [51]:
ALL_COMPLETED

'ALL_COMPLETED'

In [38]:
def evaluation(df_train, df_test):
    """
    Function that returns the average precision metric for given users.
    """
    # Real purchased products
    
    
#     print(y_real)
    # Recommended products
    popularity_outcome = popularity_based(df_train)
    actual, predicted = [], []
    users = df_test.ncodpers.unique()
    print(f'User amount: {len(users)}')
    for user_id in users:
#     hybrid_outcome = hybrid_recommend(df_train)
        print(user_id)
        y_real = rec_test(user_id, df_train, df_test)
        predicted.append(recommendation(user_id, df_train, popularity_outcome))
        actual.append(y_real)
#     print(y_pred)
    return mapk(actual, predicted)

In [18]:
df_train_0716

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
3776494,2015-07-28,952006,N,ES,V,24,2011-09-30,0.0,46,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3776495,2015-07-28,640574,N,ES,V,39,2006-09-09,0.0,106,1.0,...,0,0,0,0,0,1,0,1.0,1.0,1
3776496,2015-07-28,640570,N,ES,H,29,2006-09-09,0.0,106,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3776497,2015-07-28,640567,N,ES,V,61,2006-09-09,0.0,106,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3776498,2015-07-28,640565,N,ES,H,65,2006-09-09,0.0,106,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4606306,2015-07-28,1407968,N,ES,V,20,2015-07-17,1.0,0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4606307,2015-07-28,1407967,N,ES,H,21,2015-07-17,1.0,0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4606308,2015-07-28,1407964,N,ES,V,23,2015-07-17,1.0,0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4606309,2015-07-28,952008,N,ES,V,26,2011-09-30,0.0,46,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [19]:
df_test_0816

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
4606311,2015-08-28,753621,N,ES,H,37,2008-03-11,0.0,89,1.0,...,0,0,0,1,0,0,0,0.0,0.0,0
4606312,2015-08-28,890628,N,ES,H,48,2010-09-03,0.0,59,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4606313,2015-08-28,890627,N,ES,V,37,2010-09-03,0.0,59,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4606314,2015-08-28,890626,N,ES,H,34,2010-09-03,0.0,59,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4606315,2015-08-28,890625,N,ES,V,30,2010-09-03,0.0,59,1.0,...,0,0,0,1,0,0,0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5449507,2015-08-28,279157,N,ES,H,49,2002-10-23,0.0,110,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
5449508,2015-08-28,279156,N,ES,H,41,2001-10-05,0.0,166,1.0,...,1,0,0,1,0,1,0,0.0,0.0,1
5449509,2015-08-28,279152,N,ES,H,55,2001-10-05,0.0,166,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
5449510,2015-08-28,279183,N,ES,H,43,2001-10-05,0.0,166,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [55]:
evaluation(df_train_0716, df_test_0816[:10])

User amount: 10
753621
890628
890627
890626
890625
890622
890621
890620
890619
890618


0.0

In [57]:
NUM_PROC = 5
with ProcessPoolExecutor(max_workers=NUM_PROC) as executor:
        # Set all but one worker making salads
        futures = []
        executor._shutdown_thread = True
        for _ in range(NUM_PROC):
            futures.append(executor.submit(evaluation, df_train_0716, df_test_0816[:10]))
        done, not_done = wait(futures, return_when=ALL_COMPLETED)
        print(futures)
        for future in futures:
            print(future.result())
            


RuntimeError: cannot schedule new futures after shutdown